# Actuarial notation implemented in Python

## Motivations for actuarial notation

Remember that actuarial science predates computers. A [reference on Actuarial notation](https://www.casact.org/sites/default/files/database/proceed_proceed49_49123.pdf) traces it's origins to the 1800s:

> The existing international actuarial notation was founded on the 'Key to the
Notation' given in the Institute of Actuaries Text-Book, Part II, Life Contingencies, by George King, and is embodied in an explanatory statement
adopted unanimously by the Second International Actuarial Congress held in
London in May 1898 and printed on pp. 618-40 of the Transactions of that
Congress.

Prior to computers, actuarial calculations used in the insurance industry were done on paper. The notation was designed to be easy to write and flexible. We will largely avoid it because it is not easy to type on a computer and much of the same information can be conveyed using Python code.

## The time value of money

### Cashflows

Cashflows are money that is paid or received at a certain time. In the code below, `cashflows[i]` is the amount of money paid or received at time `i` where `i` represents the number of years since the present.

```python
cashflows = [100, 200, -50, 100, 100]
```

This is visualized in textbooks using a "time diagram".

![](./time-diagram.svg)



## Accumulated value

If you invest 1 dollar, how much money will you have in a year? The answer is more than a dollar. How much more? It depends on the interest rate. If the interest rate is 5%, then you will have $1 \cdot (1+ 0.05) = 1.05$ dollars. This is called the accumulated value of a dollar in 1 year.

* $i$ is the interest rate.
  * We assume the interest rate to be constant for this discussion.
  * If we have $1$ dollar today, we can invest it and it will become $1+i$ dollars in a year with an annual interest rate of $i$. In two years this will compound and we will have $(1+i)^2$ dollars. In $n$ years we will have $(1+i)^n$ dollars.

We can do an accumulation calculation in Python using a `for` loop, it will sum up the cashflows and project them forward with interest. The result is a list where `result[i]` is the accumulated value of the cashflows with interest at time `i`. The interest rate is assumed to be constant.

```python
cash_flows = [100, 200, -50, 100, 100]

def accumulated_value(cashflows:: list, interest_rate: float):
    result = []
    tot = 0
    for i, cf in enumerate(cashflows):
        tot += cf
        result.append(tot * (1 + interest_rate) ** i)
    return result

print(accumulated_value(cashflows, 0.05))
```

## Present value

Just like we can project cashflows forward with interest, we can also project them backward. If we have $1$ dollar in a year, how much is it worth today? Well, if we have $v$ dollars and invest it today to get $1$ dollar in a year, then $v(1+i) = 1$ and $v = \frac{1}{1+i}$. This is the present value of a dollar in 1 year.

* $v = \frac{1}{1+i}$ is the **discount factor**.
* $v$ is the **present value** of a dollar one year from now.

We can calculate the present value of cashflows.

```python
cash_flows = [100, 200, -50, 100, 100]
def get_pv_cashflows(cashflows: list, interest_rate: float):
    tot = 0
    for i, cf in enumerate(cashflows):
        tot += cf * (1 + interest_rate) ** -i
    return tot

print(get_pv_cashflows(cashflows, 0.05))
```

## Annuities

An annuity is a series of payments that are made at regular intervals. There are two types of annuities.

* Annuity-immediate: payments are made at the end of each period.
    * I have no idea why the annuity-immediate doesn't pay immediately (at the beginning of the period).
* Annuity-due: payments are made at the beginning of each period.

Here are 5 payment annuities with payments of 100 dollars. We can calculate the present value of these annuities.

```python
annuity_immediate = [0, 100, 100, 100, 100, 100]
annuity_due = [100, 100, 100, 100, 100, 0]

def get_pv_cashflows(cashflows: list, interest_rate: float):
    tot = 0
    for i, cf in enumerate(cashflows):
        tot += cf * (1 + interest_rate) ** -i
    return tot

print(f"pv_annuity_immediate: {get_pv_cashflows(annuity_immediate, 0.05)}")
print(f"pv_annuity_due: {get_pv_cashflows(annuity_due, 0.05)}")
```

## Quick maths

Actuarial exams require to be very fast at implementing these calculations by hand for special cases that have closed-form solution. Here is some of the math you might find in an introductory financial mathematics course. 

When the interest rate and payments are constant, there is a closed form for the present value of cashflows.

$$
\begin{align}
PV &= a + av + av^2 + \cdots + av^{n-1} \\
&= a \sum_{i=0}^{n-1} v^i \\
&= a \frac{1 - v^n}{1 - v}
\end{align}
$$

where $a$ is the constant payment, $i$ is the interest rate, $n$ is the number of payments, and $v = \frac{1}{1+i}$ is the discount factor.

To find the closed form for geometric series, we consider the general case for a truncated geometric series and then discuss the infinite case.

A truncated geometric series is given by:

$S_n = a + av + av^2 + \cdots + av^{n-1}$

Here, $S_n$ is the sum of the first $n$ terms, $a$ is the first term, and $r$ is the common ratio.

We can find the closed form for $S_n$ as follows:

\begin{align}
S_n &= a + av + av^2 + \cdots + av^{n-1} \\
vS_n &= av + av^2 + \cdots + av^{n-1} + av^n \\
\end{align}

Subtracting the first equation from the second equation cancels out all but the first and last terms:

\begin{align}
vS_n - S_n &= av^n - a \\
S_n(v-1) &= a(v^n - 1) \\
S_n &= a \frac{v^n - 1}{v - 1}
\end{align}

So, the closed form for a truncated geometric series is $S_n = a \frac{v^n - 1}{v - 1}$.

Quiz: Can you use the same subtraction technique from the proof to solve for the Arithmetico-geomtric series?

$$
\sum_{i=0}^{n-1} (a + id) v^i
$$